In [1]:
import pathlib
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
import tqdm
import copy

In [2]:
import timm

## Get Data Loaders

In [3]:
# Create a pytorch dataset
data_dir = pathlib.Path('/data/tiny-imagenet/')
image_count = len(list(data_dir.glob('**/*.JPEG')))
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
print('Discovered {} images'.format(image_count))

# Create the training data generator
batch_size = 32
im_height = 64
im_width = 64
num_epochs = 30

data_transforms = transforms.Compose([
    transforms.RandomCrop(64, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True, num_workers=4, pin_memory=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Discovered 120000 images


## Model

In [5]:
# model = torchvision.models.resnet50(pretrained=True)
# checkpoint = torch.load('checkpoint.pth.tar')
# model = nn.DataParallel(model)
# model.load_state_dict(checkpoint['state_dict'])
# model = model.module
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (

In [4]:
# Create a simple model
for param in list(model.parameters())[:-30]:
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features

model.fc =  nn.Sequential(
                  nn.Linear(num_ftrs, 256), 
                  nn.ReLU(),
                  nn.Linear(256, 256),
                  nn.ReLU(),
                  nn.Linear(256, 200))
model = nn.Sequential(
    nn.Linear()
)
model = model.to(device)
# model = Net(len(CLASS_NAMES), im_height, im_width)
optim = torch.optim.Adam(
    [
        {"params": list(model.parameters())[-30:-6], "lr": 1e-4},
        {"params": model.fc.parameters(), "lr": 1e-3}
    ])
criterion = nn.CrossEntropyLoss()

In [ ]:

for i in range(num_epochs):
    train_total, train_correct = 0,0
    for idx, (inputs, targets) in enumerate(train_loader):
        # print(inputs[1])
        # plt.imshow(inputs[1].permute(1,2,0))
        # plt.show()
        inputs, targets = inputs.to(device), targets.to(device)
        optim.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optim.step()
        _, predicted = outputs.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()
        # if idx % 100 == 0:
        print("\r", end='')
        print(f'training {100 * idx / len(train_loader):.2f}%: {train_correct / train_total:.3f}', end='')
    if i % 5 == 0:
        torch.save({
            'net': model.state_dict(),
        }, "results/resnet50augmix{}.pt".format(i))


##test #1:0.038 at 5.06%


training 54.69%: 0.607

## Domain Adaptation

In [6]:
# ## Test time batchnorm update with no prior

def update_bn_params(model, val_loader, num_bn_updates, gpu):
    val_loader = torch.utils.data.DataLoader(val_loader.dataset,
                                             batch_size=val_loader.batch_size,
                                             shuffle=True, num_workers=val_loader.num_workers)
    def use_test_statistics(module):
        if isinstance(module, nn.BatchNorm2d):
            module.train()
    model = copy.deepcopy(model)
    model.eval()
    model.apply(use_test_statistics)
    print("Updating BN params (num updates:{})".format(num_bn_updates))
    with torch.no_grad():
        for i, (images, target) in enumerate(val_loader):
            if i<num_bn_updates:
                images = images.cuda(gpu, non_blocking=True)
                output = model(images)
    print("Done.")
    return model


## Test Particular Model
Run to load particular model onto computer

In [13]:
model = timm.create_model('inception_resnet_v2', pretrained=True)
num_ftrs = model.classif.in_features
model.fc =  nn.Sequential(
                  nn.Dropout(0.4),
                  nn.Linear(num_ftrs, 1024), 
                  nn.ReLU(),
                  nn.Linear(1024, 256),
                  nn.ReLU(),
                  nn.Linear(256, 200))
checkpoint = torch.load('results/inceptionresnetv2_biggerFCN_linear_epoch2.pt')
model.load_state_dict(checkpoint['net'])
model = model.to(device)
# model = torchvision.models.resnet50(pretrained=True)
# num_ftrs = model.fc.in_features

# model.fc =  nn.Sequential(
#                   nn.Linear(num_ftrs, 256), 
#                   nn.ReLU(),
#                   nn.Linear(256, 256),
#                   nn.ReLU(),
#                   nn.Linear(256, 200))
# checkpoint = torch.load('resnet50-pretrainedaugmix-20epochs-nofreeze-aug.pt')
# model.load_state_dict(checkpoint['net'])
# model = model.to(device)


Run below to evaluate on validation set

In [14]:
validation_set = torchvision.datasets.ImageFolder(data_dir / 'val', transform_test)
val_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size,
                                           shuffle=True, num_workers=4, pin_memory=True)


# model = update_bn_params(model, val_loader, 32, device)
model.eval()
all_preds = []
all_labels = []
all_losses = []
with torch.no_grad():
    index = 0
    for batch in tqdm.tqdm(val_loader):
        inputs = batch[0]
        targets = batch[1]
        targets = targets.cuda()
        inputs = inputs.cuda()
        preds = model(inputs)
        loss = nn.CrossEntropyLoss()(preds, targets)
        all_losses.append(loss.cpu())
        all_preds.append(preds.cpu())
        all_labels.append(targets.cpu())

100%|██████████| 313/313 [01:15<00:00,  4.13it/s]


## Top 1 Accuracy

In [15]:
top_preds = [x.argsort(dim=-1)[:,-1:].squeeze() for x in all_preds]
correct = 0
for idx, batch_preds in enumerate(top_preds):
    correct += torch.eq(all_labels[idx], batch_preds).sum()
accuracy = correct.item() / (32 * len(all_labels))
print(f"Top 1 Validation Accuracy: {accuracy}")

Top 1 Validation Accuracy: 0.661741214057508


## Top 3 Accuracy

In [16]:
top_preds = [x.argsort(dim=-1)[:,-3:] for x in all_preds]
correct = 0
for idx, batch_preds in enumerate(top_preds):
    correct += torch.eq(all_labels[idx], batch_preds[:,0:1].squeeze()).sum()
    correct += torch.eq(all_labels[idx], batch_preds[:,1:2].squeeze()).sum()

    correct += torch.eq(all_labels[idx], batch_preds[:,2:3].squeeze()).sum()

accuracy = correct.item() / (32 * len(all_labels))
print(f"Top 3 Validation Accuracy: {accuracy}")

Top 3 Validation Accuracy: 0.8084065495207667


In [ ]:
## with BN update: top 1: 39.3% top 3: 57.96%
## without BN update

In [ ]:
# ## Test time batchnorm update with no prior

def update_bn_params(model, val_loader, num_bn_updates, gpu):
    val_loader = torch.utils.data.DataLoader(val_loader.dataset,
                                             batch_size=val_loader.batch_size,
                                             shuffle=True, num_workers=val_loader.num_workers)
    def use_test_statistics(module):
        if isinstance(module, nn.BatchNorm2d):
            module.train()
    model = copy.deepcopy(model)
    model.eval()
    model.apply(use_test_statistics)
    print("Updating BN params (num updates:{})".format(num_bn_updates))
    with torch.no_grad():
        for i, (images, target) in enumerate(val_loader):
            if i<num_bn_updates:
                images = images.cuda(gpu, non_blocking=True)
                output = model(images)
    print("Done.")
    return model


## Test Particular Model
Run to load particular model onto computer

model = torchvision.models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

model.fc =  nn.Sequential(
                  nn.Linear(num_ftrs, 256), 
                  nn.ReLU(),
                  nn.Linear(256, 256),
                  nn.ReLU(),
                  nn.Linear(256, 200))
checkpoint = torch.load('resnet50-pretrainedaugmix-20epochs-nofreeze-aug.pt')
model.load_state_dict(checkpoint['net'])
model = model.to(device)


Run below to evaluate on validation set

validation_set = torchvision.datasets.ImageFolder(data_dir / 'val', transform_test)
val_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size,
                                           shuffle=True, num_workers=4, pin_memory=True)


model = update_bn_params(model, val_loader, 128, device)
model.eval()
all_preds = []
all_labels = []
all_losses = []
with torch.no_grad():
    index = 0
    for batch in tqdm.tqdm(val_loader):
        inputs = batch[0]
        targets = batch[1]
        targets = targets.cuda()
        inputs = inputs.cuda()
        preds = model(inputs)
        loss = nn.CrossEntropyLoss()(preds, targets)
        all_losses.append(loss.cpu())
        all_preds.append(preds.cpu())
        all_labels.append(targets.cpu())

## Top 1 Accuracy

top_preds = [x.argsort(dim=-1)[:,-1:].squeeze() for x in all_preds]
correct = 0
for idx, batch_preds in enumerate(top_preds):
    correct += torch.eq(all_labels[idx], batch_preds).sum()
accuracy = correct.item() / (32 * len(all_labels))
print(f"Top 1 Validation Accuracy: {accuracy}")

## Top 3 Accuracy

top_preds = [x.argsort(dim=-1)[:,-3:] for x in all_preds]
correct = 0
for idx, batch_preds in enumerate(top_preds):
    correct += torch.eq(all_labels[idx], batch_preds[:,0:1].squeeze()).sum()
    correct += torch.eq(all_labels[idx], batch_preds[:,1:2].squeeze()).sum()

    correct += torch.eq(all_labels[idx], batch_preds[:,2:3].squeeze()).sum()

accuracy = correct.item() / (32 * len(all_labels))
print(f"Top 3 Validation Accuracy: {accuracy}")

## with BN update: top 1: 39.3% top 3: 57.96%
## without BN update